In [ ]:
# Weighted Interval Scheduling Algorithm Using
#  Recursive "top-down" Dynamic Programming
# Author: Dr. Jordan Malof, University of Montana
# Course: CSCI 332, Fall 2023
# Date:  10/29/2023
#
# INSTRUCTIONS: Below is a skeleton of a  Dynamic programming algorithm to
# solve the weighted interval scheduling problem in Chapter 6.1 of the
# Kleinberg-Tardos texbook.  We will be implementing the "top-down"
# recursive form of the algorithm that is discussed in Chapter 6.1, as opposed
# to the "bottom-up" version in Chapter 6.2.
#
# We will work with numpy arrays in this exercise because numpy has some
#  useful functions that will make the job a bit easier.
#
#
# Needed libraries
import numpy as np
import matplotlib.pyplot as plt
import random

# import debugger libraries (OPTIONAL)
# !pip install -Uqq ipdb
# import ipdb

# Uncommont this to turn debugger on.
# %pdb on
# Uncomment this to turn debugger off
# %pdb off

# Remember you can write ipdb.set_trace() to set a breakpoint
# after turning the debugger on

In [ ]:
#
#  Some test input interval sets below
#  A list of intervals, where each entry in the list
#  contains the start time (s_i), finish time (f_i), and value (v_i)
#  for the i^th interval,
#   i.e.,  interval_list[i] = [s_i, f_i, v_i]
#
#  For simplicity, we assume these examples are already ordered by
#   finish time.
#
#
# ##########################################
# ####### A Harder Example  Still     ####
# ##########################################

# # adjaceny connectivity list for graph
# interval_list = np.array([[1,3,1],
#                   [0.5,3.5,8],
#                   [2,4,4],
#                   [3,5,2]])
#
# m = [1,8,8,8]
# optimal_indices = [1]
#
# ##########################################
# ####### Slightly Harder EXAMPLE       ####
# ##########################################

# adjaceny connectivity list for graph
# interval_list = np.array([[1,3,1],
#                  [2,4,4],
#                 [3,5,2],])

# What should the optimal solutio be?
#  m = [1,4,4]
#  optimal_indices = [1]

##########################################
####### SIMPLE INPUT EXAMPLE       ####
##########################################

# A list of intervals, with [s_i, f_i, v_i]
interval_list = np.array([[1,3,1],
                [2,4,1],
                [3,5,1],])
#
# What should the optimal solution be?
# m = [1,1,2]
# optimal_indices = [0,2];
#


In [ ]:
# TEST YOUR CODE
#  NOTE: Don't forget to execute the blocks with
#  the other functions in them!
#
m,optimal_indices = my_scheduler(interval_list);
print(m)
print(optimal_indices)

In [ ]:
# IMPLEMENTATION of dynamic programming algorithm to find the optimal subset
#  of intervals in 'interval_list' to solve the weighted interval scheduling
#  problem.  This returns the optimal value 'm', and the indices of the
#  intervals.
#
# INPUT:
#     sorted_interval_list - an nx3 list with the start time, finish time, and weight
#         of each interval.  We assume that the input is already sorted by
#          finish time.
#
# OUTPUT
#     m - the solution to each optimization problem
#     optimal_indices - indices of intervals in the optimal solution

def my_scheduler(sorted_interval_list):

  #Get number of intervals in set
  n = sorted_interval_list.shape[0];

  # Compute the 'p_ind' list
  p_ind = -1*np.ones(n,dtype=np.int8)
  for i in np.arange(n):
    start_time = sorted_interval_list[i,0];

    # Working backwards in finish time, find first interval that finishes
    # before the current interval starts
    for ii in np.arange(i-1,-1,-1):
      if sorted_interval_list[ii,1]<=start_time:
        p_ind[i]=ii;
        break;

  # Initialize list of optimal values
  m = np.zeros(n,dtype=np.int8);

  # Get list of interval values, v_i
  v = sorted_interval_list[:,2]

  # Run the recursion to computer optimal solutions
  val,m = m_compute_opt(n-1,m,p_ind,v);

  # Find the solution
  optimal_indices = np.array([])
  optimal_indices = find_solution(n-1, m, p_ind,v,optimal_indices)

  # Return results
  # return optimal_indices,m
  return m,optimal_indices

In [ ]:
# IMPLEMENTATION of recursive dynamic programming algorithm to find
#  the optimal solution to all sub-problems
#  Assume we input an n-length adjacency list, 'A', representing a graph, and
#  an adjency list, 'W', representing the weight of each directed edge.
#  We then return a list 'd' indicating the shortest distance to each node
#  in the input graph
#
# INPUT:
#     j - index of sorted intervals that we are considering.  Assumes intervals
#         have been sorted by their finish times
#     m - array of values of optimal solutions, as a function of the interval
#         index for which we are solving the problem.
#     p_ind - array of indices indicating, for each interval, which interval in
#         the set has the latest finish time while still not conflicting with
#         the start time of that interval
#     v - values/weights of each interval
#
# OUTPUT
#     m - the optimal value of the interval scheduling problem of each size,
#            with solutions filled in from at least 1...j.
#

def m_compute_opt(i,m,p_ind,v):
  # ipdb.set_trace()
  #If we are at i=0, then by definition optimal solution is 0
  if i==-1:
    val = 0;
    return val,m;

  # Check if the solution already exists
  #  if it does, then just retun it
  elif m[i]!=0:
    val = m[i]
    return val,m;   # We don't need to do anything

  # If optimal solution doesn't already exist, compute it
  else:
    # Call function recursively
    # Function has to return 'm', the whole array.  Then we pull out the
    #  value that we need
    o_with_inclusion,m = m_compute_opt(p_ind[i],m,p_ind,v);
    o_without_inclusion,m = m_compute_opt(i-1,m,p_ind,v)

    # Computer the optimal solution for the i^th entry
    m[i] = max(v[i]+o_with_inclusion,o_without_inclusion)
    val = m[i]
    return val,m

In [ ]:
#     optimal_list - the indices of intervals included in the optimal solution
#
def find_solution(i, m, p_ind,v,optimal_indices):

  if i<0:
    optimal_indices = optimal_indices;

  else:

    # Must be careful with p_ind
      if p_ind[i]==-1:
          val_add=0;
      else:
          val_add = m[p_ind[i]];

      #Compute optimal value if
      o_with = v[i]+val_add;

      if i-1<0:
        o_without = 0;
      else:
        o_without = m[i-1]

      # If including interval 'index' is better, include it
      if o_with > o_without:
        indices_to_add = find_solution(p_ind[i],m,p_ind,v,optimal_indices)
        optimal_indices = np.concatenate((optimal_indices,indices_to_add,np.array([i],dtype=np.int8))) # adding array i - this is the index of the interval we are adding
      else:
        indices_to_add = find_solution(i-1,m,p_ind,v,optimal_indices)
        optimal_indices = np.concatenate((optimal_indices,indices_to_add))


  return optimal_indices